# Project Capstone - Segmenting and Clustering Neighborhoods - Toronto  Assignment

## This notebook includes 3 parts 
* **Part 1 - Reading the data from the web link of postal codes of Canada.**
* **Part 2 - Reading the Geospatial details of the postal codes from the CSV file provided.**
* **Part 3 - Explore and cluster the Neighbourhood of Toronto, Canada.**

## Part 1 - 
**Read the table from the web link https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M using the pandas Library**

In [1]:
# importing the required libraries
import pandas as pd
import numpy as np

**Assign the Url to a variable and read using read_html.**

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df_web = pd.read_html(url)  # this will return a list..
print("Type of df_web :", type(df_web))

#To get the dataframe we will read the first occurence of the lis, which is a dataframe
df_WEB = df_web[0]
print("Type of df_WEB :", type(df_WEB))

Type of df_web : <class 'list'>
Type of df_WEB : <class 'pandas.core.frame.DataFrame'>


**Check the first few rows of the dataframe.**

In [3]:
df = df_WEB.copy()
df.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
8,M8A,Not assigned,Not assigned
9,M9A,Etobicoke,Islington Avenue


**Clean the dataframe rows with value of "not assigned".**

In [4]:
df.Borough.replace(to_replace='Not assigned',value=np.nan,inplace=True)
df.Neighbourhood.replace(to_replace='Not assigned',value=np.nan,inplace=True)
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [5]:
df.dropna(inplace=True)

In [6]:
df.reset_index(drop=True).head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


In [7]:
df_WEB.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [8]:
print("Total number of rows in original dataset were {}. {} number of 'Not Assigned' rows were removed from the original dataset.".format(df_WEB.shape[0],df_WEB.shape[0]-df.shape[0]))

Total number of rows in original dataset were 287. 77 number of 'Not Assigned' rows were removed from the original dataset.


In [9]:
df.dtypes

Postcode         object
Borough          object
Neighbourhood    object
dtype: object

## Part 2.
**Include the Latitude and longitude for the Neighbourhoods from the csv file http://cocl.us/Geospatial_data**

In [10]:
url1 = 'http://cocl.us/Geospatial_data'
df_geo = pd.read_csv(url1)
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


**Merge the two files to include the latitude and longitude for each location**

In [11]:
df3 = df.merge(df_geo,left_on="Postcode",right_on="Postal Code",how='left')
df3.head()

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,M5A,43.654260,-79.360636
3,M6A,North York,Lawrence Heights,M6A,43.718518,-79.464763
4,M6A,North York,Lawrence Manor,M6A,43.718518,-79.464763


In [12]:
df3.drop(columns="Postal Code",axis=1,inplace=True)

In [13]:
df3[df3['Postcode']=='M5T']

,Postcode,Borough,Neighbourhood,Latitude,Longitude
151,M5T,Downtown Toronto,Chinatown,43.653206,-79.400049
152,M5T,Downtown Toronto,Grange Park,43.653206,-79.400049
153,M5T,Downtown Toronto,Kensington Market,43.653206,-79.400049


In [14]:
df3.shape

(210, 5)

In [15]:
df3.to_csv('Tor_city_Neighbour_Lat_Lng.csv',encoding='utf-8',index=False)

## Part - 3.
**Explore and cluster the neighborhoods in Toronto. This includes the boroughs that contain the word Toronto.**

* **Map the Neighbourhoods**

In [15]:
df4=[]
str = 'Toronto'
for n in range(0,df3.shape[0]):
    if str in df3.iloc[n,1]:
        df4.append(df3.iloc[n,:])

In [16]:
df5 = pd.DataFrame(df4)
df5.reset_index(drop=True,inplace=True)
df5.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
1,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
2,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937
3,M5B,Downtown Toronto,Garden District,43.657162,-79.378937
4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418


In [17]:
df5.shape

(74, 5)

In [18]:
import folium 
from geopy.geocoders import Nominatim

In [20]:
address = 'Toronto, ON'
geolocator = Nominatim(user_agent='ny_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical cordinates of Toronto city are {},{}'.format(latitude,longitude))

The geographical cordinates of Toronto city are 43.653963,-79.387207


In [21]:
map_Tor = folium.Map(location=[latitude,longitude],zoom_start=10)

# add markers to the map
for lat, lng, borough, neighborhood in zip(df5['Latitude'],df5['Longitude'],df5['Borough'],df5['Neighbourhood']):
    label = '{},{}'.format(neighborhood,borough)
    label = folium.Popup(label,parse_html=True)
    folium.CircleMarker(
            [lat,lng],radius=5,popup=label,color='blue',fill=True,fill_color='#3186cc',fill_opacity=0.7,
             parse_html=False).add_to(map_Tor)
map_Tor

In [22]:
print("There are {} Borough and {} Neighbourhoods.".format(df5.Borough.nunique(),df5.shape[0]))

There are 4 Borough and 74 Neighbourhoods.


* **Explore with clusters**

In [23]:
# Four square credentials
CLIENT_ID = '50UMKE0OQMLO2NQF0ERLPROW4J42AA2TTQ0SZJ2S230H3KIO'
CLIENT_SECRET = '5E0ML3XE2KKTLCAIQJVQLEKQ3NAFCYGW1U31FEQ3DRVPIHM2'
VERSION = '20200309'
RADIUS = 500
LIMIT = 100
print("Credentials are: \n"+"Clinet_ID :-"+CLIENT_ID+"\nClient_Secret :-"+CLIENT_SECRET,"\nVersion :"+VERSION)

Credentials are: 
Clinet_ID :-50UMKE0OQMLO2NQF0ERLPROW4J42AA2TTQ0SZJ2S230H3KIO
Client_Secret :-5E0ML3XE2KKTLCAIQJVQLEKQ3NAFCYGW1U31FEQ3DRVPIHM2 
Version :20200309


In [42]:
# Define a list to get the nearby Venues details using foursqaure request
 
import requests
venues_list=[]
for pc,br,name,lat,lng in zip(df5['Postcode'],df5['Borough'],df5['Neighbourhood'],df5['Latitude'],df5['Longitude']):
    print(pc,br,name)
    
    #Create the request
    url2 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID,CLIENT_SECRET,VERSION,lat,lng,RADIUS,LIMIT)
    
    #Get request
    results = requests.get(url2).json()['response']['groups'][0]['items']
        
    #extract the relevant fields required from the get request
    for venue in results:
        venues_list.append([(pc,
                             br,
                             name,
                             lat,
                             lng,
                             venue['venue']['name'],
                             venue['venue']['location']['lat'],
                             venue['venue']['location']['lng'],
                             venue['venue']['categories'][0]['name'])])

M5A Downtown Toronto Harbourfront
M7A Downtown Toronto Queen's Park
M5B Downtown Toronto Ryerson
M5B Downtown Toronto Garden District
M5C Downtown Toronto St. James Town
M4E East Toronto The Beaches
M5E Downtown Toronto Berczy Park
M5G Downtown Toronto Central Bay Street
M6G Downtown Toronto Christie
M5H Downtown Toronto Adelaide
M5H Downtown Toronto King
M5H Downtown Toronto Richmond
M6H West Toronto Dovercourt Village
M6H West Toronto Dufferin
M5J Downtown Toronto Harbourfront East
M5J Downtown Toronto Toronto Islands
M5J Downtown Toronto Union Station
M6J West Toronto Little Portugal
M6J West Toronto Trinity
M4K East Toronto The Danforth West
M4K East Toronto Riverdale
M5K Downtown Toronto Design Exchange
M5K Downtown Toronto Toronto Dominion Centre
M6K West Toronto Brockton
M6K West Toronto Exhibition Place
M6K West Toronto Parkdale Village
M4L East Toronto The Beaches West
M4L East Toronto India Bazaar
M5L Downtown Toronto Commerce Court
M5L Downtown Toronto Victoria Hotel
M4M Eas

In [43]:
Venues_df = pd.DataFrame(venues_list)
type(Venues_df)

pandas.core.frame.DataFrame

In [44]:
Venues_df = pd.DataFrame(item for venue_list in venues_list for item in venue_list)

In [45]:
Venues_df.columns = ['Postcode','Borough','Neighbourhood','N_Latitude','N_Longitude','Venue','V_Latitude','V_Longitude','V_Category']

In [46]:
Venues_df.head()

,Postcode,Borough,Neighbourhood,N_Latitude,N_Longitude,Venue,V_Latitude,V_Longitude,V_Category
0,M5A,Downtown Toronto,Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,M5A,Downtown Toronto,Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,M5A,Downtown Toronto,Harbourfront,43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,M5A,Downtown Toronto,Harbourfront,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,M5A,Downtown Toronto,Harbourfront,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


In [47]:
# Number of Venues in each Neighbouthood
Venues_df.groupby('Neighbourhood').count().sort_values(by='Postcode',ascending=False)

,Postcode,Borough,N_Latitude,N_Longitude,Venue,V_Latitude,V_Longitude,V_Category
Neighbourhood,,,,,,,,
St. James Town,146,146,146,146,146,146,146,146
Adelaide,100,100,100,100,100,100,100,100
Garden District,100,100,100,100,100,100,100,100
Harbourfront East,100,100,100,100,100,100,100,100
First Canadian Place,100,100,100,100,100,100,100,100
King,100,100,100,100,100,100,100,100
Richmond,100,100,100,100,100,100,100,100
Design Exchange,100,100,100,100,100,100,100,100
Ryerson,100,100,100,100,100,100,100,100


* **St James Town has the highest number of Venues**

In [48]:
Venues_df['V_Category'].nunique()

232

In [49]:
Venues_df_En = pd.concat([Venues_df['Neighbourhood'],pd.get_dummies(Venues_df['V_Category'],drop_first=True)],axis=1)
Venues_df_En.shape

(3245, 232)

In [50]:
Venues_df_En.head(10)

,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [51]:
Toronto_Venues_grp = Venues_df_En.groupby(by='Neighbourhood').mean().reset_index()
Toronto_Venues_grp.shape

(73, 232)

In [52]:
Toronto_Venues_grp.head(10)

,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Adelaide,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.0,0.0,...,0.0,0.0,0.020000,0.0,0.00000,0.010000,0.0,0.0,0.01,0.000000
1,Bathurst Quay,0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.00000,0.000000,0.0,0.0,0.00,0.000000
2,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.017857,0.0,0.00000,0.000000,0.0,0.0,0.00,0.000000
3,Brockton,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.00000,0.000000,0.0,0.0,0.00,0.041667
4,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.00000,0.000000,0.0,0.0,0.00,0.058824
5,CN Tower,0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.00000,0.000000,0.0,0.0,0.00,0.000000
6,Cabbagetown,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.021739,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.00000,0.000000,0.0,0.0,0.00,0.000000
7,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012346,0.0,0.0,...,0.0,0.0,0.012346,0.0,0.00000,0.012346,0.0,0.0,0.00,0.012346
8,Chinatown,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.034884,0.0,0.05814,0.011628,0.0,0.0,0.00,0.000000
9,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.00000,0.000000,0.0,0.0,0.00,0.000000


In [53]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
cols = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        cols.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        cols.append('{}th Most Common Venue'.format(ind+1))
        
dataframe_Tor = pd.DataFrame(columns=cols)
dataframe_Tor['Neighbourhood'] = Toronto_Venues_grp['Neighbourhood']
        
for Neighbour in Toronto_Venues_grp['Neighbourhood']:
#     print('----'+Neighbour+'----')
    temp = Toronto_Venues_grp[Toronto_Venues_grp['Neighbourhood'] == Neighbour].T.reset_index()
    temp.columns = ['venue', 'freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    temp1 = temp.sort_values('freq', ascending = False).reset_index(drop=True).head(num_top_venues)
    indx = Toronto_Venues_grp[Toronto_Venues_grp['Neighbourhood']== Neighbour].index.values.astype('int')[0]
#     print('\n')
    Cat_names = temp1['venue'].to_list()
    dataframe_Tor.iloc[indx,:].iloc[1:] = Cat_names
dataframe_Tor.head(10)

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Restaurant,Café,Thai Restaurant,Sushi Restaurant,Bar,Gym,Lounge,Breakfast Spot,Steakhouse
1,Bathurst Quay,Airport Service,Airport Lounge,Airport Terminal,Airport,Rental Car Location,Boutique,Boat or Ferry,Sculpture Garden,Coffee Shop,Bar
2,Berczy Park,Coffee Shop,Café,Seafood Restaurant,Farmers Market,Beer Bar,Restaurant,Cheese Shop,Cocktail Bar,Bakery,Breakfast Spot
3,Brockton,Café,Breakfast Spot,Coffee Shop,Bar,Burrito Place,Climbing Gym,Convenience Store,Furniture / Home Store,Grocery Store,Gym
4,Business Reply Mail Processing Centre 969 Eastern,Yoga Studio,Garden Center,Pizza Place,Comic Shop,Recording Studio,Restaurant,Farmers Market,Fast Food Restaurant,Skate Park,Burrito Place
5,CN Tower,Airport Service,Airport Lounge,Airport Terminal,Airport,Rental Car Location,Boutique,Boat or Ferry,Sculpture Garden,Coffee Shop,Bar
6,Cabbagetown,Restaurant,Café,Coffee Shop,Pub,Pizza Place,Italian Restaurant,Bakery,Caribbean Restaurant,Liquor Store,Breakfast Spot
7,Central Bay Street,Coffee Shop,Italian Restaurant,Japanese Restaurant,Sandwich Place,Ice Cream Shop,Juice Bar,Burger Joint,Café,Middle Eastern Restaurant,Salad Place
8,Chinatown,Bar,Vietnamese Restaurant,Bakery,Café,Coffee Shop,Chinese Restaurant,Mexican Restaurant,Vegetarian / Vegan Restaurant,Dumpling Restaurant,Farmers Market
9,Christie,Grocery Store,Café,Coffee Shop,Park,Gas Station,Italian Restaurant,Nightclub,Restaurant,Baby Store,Candy Store


**Create the clusters**

In [54]:
# import yhr library
from sklearn.cluster import KMeans
KM = KMeans(n_clusters=5,init='k-means++')

In [55]:
dataframe_Tor.shape

(73, 11)

In [56]:
# prepare the features to cluster
x = Toronto_Venues_grp.iloc[:,1:237]
model = KM.fit(x)
model.labels_

array([2, 3, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 0, 4,
       0, 2, 2, 2, 2, 2, 3, 2, 2, 3, 2, 2, 3, 0, 2, 1, 2, 2, 2, 2, 2, 3,
       4, 2, 2, 2, 0, 2, 2, 2, 4, 3, 2, 2, 2, 1, 4, 2, 2, 4, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2])

In [57]:
dataframe_Tor['Cluster'] = model.labels_

In [58]:
dataframe_Tor.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster
0,Adelaide,Coffee Shop,Restaurant,Café,Thai Restaurant,Sushi Restaurant,Bar,Gym,Lounge,Breakfast Spot,Steakhouse,2
1,Bathurst Quay,Airport Service,Airport Lounge,Airport Terminal,Airport,Rental Car Location,Boutique,Boat or Ferry,Sculpture Garden,Coffee Shop,Bar,3
2,Berczy Park,Coffee Shop,Café,Seafood Restaurant,Farmers Market,Beer Bar,Restaurant,Cheese Shop,Cocktail Bar,Bakery,Breakfast Spot,2
3,Brockton,Café,Breakfast Spot,Coffee Shop,Bar,Burrito Place,Climbing Gym,Convenience Store,Furniture / Home Store,Grocery Store,Gym,2
4,Business Reply Mail Processing Centre 969 Eastern,Yoga Studio,Garden Center,Pizza Place,Comic Shop,Recording Studio,Restaurant,Farmers Market,Fast Food Restaurant,Skate Park,Burrito Place,2


In [59]:
DF_C0 = dataframe_Tor[dataframe_Tor['Cluster']==0]
DF_C1 = dataframe_Tor[dataframe_Tor['Cluster']==1]
DF_C2 = dataframe_Tor[dataframe_Tor['Cluster']==2]
DF_C3 = dataframe_Tor[dataframe_Tor['Cluster']==3]
DF_C4 = dataframe_Tor[dataframe_Tor['Cluster']==4]

**Examine the clusters**

In [60]:
print(DF_C0.shape)
DF_C0.head()

(4, 12)


,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster
20,Forest Hill North,Park,Jewelry Store,Trail,Sushi Restaurant,Airport,Movie Theater,Market,Massage Studio,Mediterranean Restaurant,Men's Store,0
22,Forest Hill West,Park,Jewelry Store,Trail,Sushi Restaurant,Airport,Movie Theater,Market,Massage Studio,Mediterranean Restaurant,Men's Store,0
35,Lawrence Park,Park,Bus Line,Swim School,Airport,Movie Theater,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,0
48,Rosedale,Park,Playground,Trail,Museum,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,0


In [61]:
print(DF_C1.shape)
DF_C1.head()

(2, 12)


,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster
37,Moore Park,Playground,Trail,Music Venue,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,1
57,Summerhill East,Playground,Trail,Music Venue,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,1


In [62]:
print(DF_C2.shape)
DF_C2.head()

(54, 12)


,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster
0,Adelaide,Coffee Shop,Restaurant,Café,Thai Restaurant,Sushi Restaurant,Bar,Gym,Lounge,Breakfast Spot,Steakhouse,2
2,Berczy Park,Coffee Shop,Café,Seafood Restaurant,Farmers Market,Beer Bar,Restaurant,Cheese Shop,Cocktail Bar,Bakery,Breakfast Spot,2
3,Brockton,Café,Breakfast Spot,Coffee Shop,Bar,Burrito Place,Climbing Gym,Convenience Store,Furniture / Home Store,Grocery Store,Gym,2
4,Business Reply Mail Processing Centre 969 Eastern,Yoga Studio,Garden Center,Pizza Place,Comic Shop,Recording Studio,Restaurant,Farmers Market,Fast Food Restaurant,Skate Park,Burrito Place,2
6,Cabbagetown,Restaurant,Café,Coffee Shop,Pub,Pizza Place,Italian Restaurant,Bakery,Caribbean Restaurant,Liquor Store,Breakfast Spot,2


In [63]:
print(DF_C3.shape)
DF_C3.head()

(7, 12)


,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster
1,Bathurst Quay,Airport Service,Airport Lounge,Airport Terminal,Airport,Rental Car Location,Boutique,Boat or Ferry,Sculpture Garden,Coffee Shop,Bar,3
5,CN Tower,Airport Service,Airport Lounge,Airport Terminal,Airport,Rental Car Location,Boutique,Boat or Ferry,Sculpture Garden,Coffee Shop,Bar,3
28,Harbourfront West,Airport Service,Airport Lounge,Airport Terminal,Airport,Rental Car Location,Boutique,Boat or Ferry,Sculpture Garden,Coffee Shop,Bar,3
31,Island airport,Airport Service,Airport Lounge,Airport Terminal,Airport,Rental Car Location,Boutique,Boat or Ferry,Sculpture Garden,Coffee Shop,Bar,3
34,King and Spadina,Airport Service,Airport Lounge,Airport Terminal,Airport,Rental Car Location,Boutique,Boat or Ferry,Sculpture Garden,Coffee Shop,Bar,3


In [64]:
print(DF_C4.shape)
DF_C4.head()

(6, 12)


,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster
14,Deer Park,Pub,Coffee Shop,Light Rail Station,Bagel Shop,Pizza Place,Restaurant,Liquor Store,Sports Bar,Fried Chicken Joint,Supermarket,4
21,Forest Hill SE,Pub,Coffee Shop,Light Rail Station,Bagel Shop,Pizza Place,Restaurant,Liquor Store,Sports Bar,Fried Chicken Joint,Supermarket,4
44,Rathnelly,Pub,Coffee Shop,Light Rail Station,Bagel Shop,Pizza Place,Restaurant,Liquor Store,Sports Bar,Fried Chicken Joint,Supermarket,4
52,South Hill,Pub,Coffee Shop,Light Rail Station,Bagel Shop,Pizza Place,Restaurant,Liquor Store,Sports Bar,Fried Chicken Joint,Supermarket,4
58,Summerhill West,Pub,Coffee Shop,Light Rail Station,Bagel Shop,Pizza Place,Restaurant,Liquor Store,Sports Bar,Fried Chicken Joint,Supermarket,4


### Observation
**The Cluster 2 has a highest number of Neighbourhoods with most business area like Resturants, Breakfast spots, Bar and Pubs.**